## Классификация текстов: спам-фильтр для SMS

In [1]:
def write_answer_to_file(answer, filename, round_to=None):
    with open(filename, 'w') as f_out:
        f_out.write(str(
            answer if round_to == None else round(answer, round_to)
        ))

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

Загрузите датасет. Описание датасета можно посмотреть [здесь](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection).

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

In [3]:
df = pd.read_csv(r'SMSSpamCollection.txt', sep='	', header=None)
df.columns = ['target', 'text']
df.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.target = df.target.map({'ham': 0, 'spam': 1})
print('Nan: {}'.format(df.isna().sum().sum()))
print('Shape: {}'.format(df.shape))
df.head()

Nan: 0
Shape: (5572, 2)


,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [5]:
vectorizer = CountVectorizer()
feature_matrix = vectorizer.fit_transform(df.text).toarray()

In [6]:
feature_matrix.shape

(5572, 8713)

Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой

In [7]:
scores = cross_val_score(LogisticRegression(random_state=2, solver="lbfgs"), feature_matrix, df.target, scoring='f1', cv=10)
mean_score = scores.mean()

In [8]:
mean_score

0.9311269283144492

In [9]:
write_answer_to_file(mean_score, '1.txt', 1)

 А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:
* *"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"*

* *"FreeMsg: Txt: claim your reward of 3 hours talk time"*

* *"Have you visited the last lecture on physics?"*

* *"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"*

* *"Only 99$"*

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [10]:
predict_list = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB", 
              "FreeMsg: Txt: claim your reward of 3 hours talk time", 
              "Have you visited the last lecture on physics?", 
              "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$", 
              "Only 99$"]

In [11]:
clf = Pipeline(
    [('vectorizer', CountVectorizer()), 
     ('classifier', LogisticRegression(random_state=2, solver="lbfgs"))
    ])

clf.fit(df.text, df.target)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ..., penalty='l2', random_state=2, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

In [12]:
predicts = clf.predict(predict_list)
predicts

array([1, 1, 0, 0, 0], dtype=int64)

In [13]:
write_answer_to_file(" ".join(str(x) for x in predicts), '2.txt')

Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [14]:
def test_ngram(ngram_range, classifier):
    clf_with_ngram = Pipeline(
        [('vectorizer', CountVectorizer(ngram_range=ngram_range)), 
         ('classifier', classifier)
        ])
    scores = cross_val_score(clf_with_ngram, df.text, df.target, scoring='f1', cv=10)
    return round(scores.mean(), 2)

In [15]:
loreg_clf = LogisticRegression(random_state=2, solver="lbfgs")
answer_logreg = [
    test_ngram((2,2), loreg_clf),
    test_ngram((3,3), loreg_clf),
    test_ngram((1,3), loreg_clf)]
answer_logreg

[0.82, 0.73, 0.92]

In [16]:
write_answer_to_file(" ".join(str(x) for x in answer_logreg), '3.txt')

Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [17]:
NB_clf = MultinomialNB()
answer_NB = [
    test_ngram((2,2), NB_clf),
    test_ngram((3,3), NB_clf),
    test_ngram((1,3), NB_clf)]
answer_NB

[0.93, 0.87, 0.95]

In [18]:
write_answer_to_file(" ".join(str(x) for x in answer_NB), '4.txt')

Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данны

In [19]:
clf_with_Tfidf = Pipeline(
    [('vectorizer', TfidfVectorizer()), 
     ('classifier', loreg_clf)
    ])
scores = cross_val_score(clf_with_Tfidf, df.text, df.target, scoring='f1', cv=10)
scores.mean()

0.8785100455343396

In [20]:
write_answer_to_file("-1", '5.txt')